In [ ]:
!pip install facenet-pytorch

     |████████████████████████████████| 1.9 MB 5.1 MB/s 


In [ ]:
# import tarfile
# lfw = tarfile.open('lfw.tgz')
# lfw.extractall('LFW')

In [ ]:
import sys
import os.path as path
import os

classes = os.listdir('./LFW/lfw')
# num_images = []
# for i in classes:
#   num_images.append(len(os.listdir('./LFW/lfw/{0}'.format(i))))

In [ ]:
import pandas as pd
df = pd.DataFrame({'class':classes,'num':num_images})

NameError: ignored

In [ ]:
labels = []
paths = []
for i in classes:
  files = os.listdir("./LFW/lfw/{0}".format(i))
  paths_i = ["./LFW/lfw/{0}/{1}".format(i,j) for j in files]
  paths = paths + paths_i
  labels = labels + [i]*len(paths_i)


In [ ]:
df = pd.DataFrame({'label':labels, 'path': paths})
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit(df['label'])

LabelEncoder()

In [ ]:
df['label_encoded'] = le.transform(df['label'])

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1

# If required, create a face detection pipeline using MTCNN:
mtcnn = MTCNN(image_size=250, margin=40)

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(classify=True, num_classes = len(classes),pretrained='casia-webface')

In [ ]:
batch_size = 16

In [ ]:
df.sample(frac=1)

,label,path,label_encoded
2144,Billy_Edelin,./LFW/lfw/Billy_Edelin/Billy_Edelin_0001.jpg,572
10643,Daniel_Scioli,./LFW/lfw/Daniel_Scioli/Daniel_Scioli_0001.jpg,1148
10326,Alan_Greenspan,./LFW/lfw/Alan_Greenspan/Alan_Greenspan_0002.jpg,93
12676,AnFernce_Negron,./LFW/lfw/AnFernce_Negron/AnFernce_Negron_0001...,235
387,Abdullah_Gul,./LFW/lfw/Abdullah_Gul/Abdullah_Gul_0014.jpg,20
...,...,...,...
857,Elvis_Costello,./LFW/lfw/Elvis_Costello/Elvis_Costello_0001.jpg,1563
3068,Jean_Chretien,./LFW/lfw/Jean_Chretien/Jean_Chretien_0023.jpg,2468
10476,Denzel_Washington,./LFW/lfw/Denzel_Washington/Denzel_Washington_...,1314
1995,James_Williams,./LFW/lfw/James_Williams/James_Williams_0001.jpg,2360


In [ ]:
from torch.nn import CrossEntropyLoss
import cv2
import torch
from torch.optim import Adam
from tqdm.notebook import tqdm
import numpy as np

criterion = CrossEntropyLoss()
optimizer = Adam(resnet.parameters())

for i in tqdm(range(len(df)//batch_size)):
  batch = df.loc[i:i+batch_size]
  images = [cv2.imread(path) for path in batch['path']]
  images = [mtcnn(img) if mtcnn(img)!=None else torch.from_numpy(np.swapaxes(img, 0, -1)) for img in images]

  optimizer.zero_grad()
  images = torch.stack(images)
  preds = resnet(images)
  labels = torch.from_numpy(le.transform(np.array(list(batch['label']))))
  loss = criterion(preds, labels)

  print(loss.mean())
  loss.backward()
  optimizer.step()


  0%|          | 0/827 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


tensor(4.4229, grad_fn=<MeanBackward0>)
tensor(0.8451, grad_fn=<MeanBackward0>)
tensor(0.6414, grad_fn=<MeanBackward0>)
tensor(0.3716, grad_fn=<MeanBackward0>)
tensor(0.3343, grad_fn=<MeanBackward0>)
tensor(0.1949, grad_fn=<MeanBackward0>)
tensor(0.3054, grad_fn=<MeanBackward0>)
tensor(0.5344, grad_fn=<MeanBackward0>)
tensor(0.9541, grad_fn=<MeanBackward0>)
tensor(1.0736, grad_fn=<MeanBackward0>)
tensor(1.2193, grad_fn=<MeanBackward0>)
tensor(1.1739, grad_fn=<MeanBackward0>)
tensor(1.1037, grad_fn=<MeanBackward0>)
tensor(1.1161, grad_fn=<MeanBackward0>)
tensor(1.5403, grad_fn=<MeanBackward0>)
tensor(2.4354, grad_fn=<MeanBackward0>)
tensor(2.4481, grad_fn=<MeanBackward0>)
tensor(2.8006, grad_fn=<MeanBackward0>)
tensor(3.3592, grad_fn=<MeanBackward0>)
tensor(3.6305, grad_fn=<MeanBackward0>)
tensor(3.7230, grad_fn=<MeanBackward0>)
tensor(3.7215, grad_fn=<MeanBackward0>)
tensor(4.0194, grad_fn=<MeanBackward0>)
tensor(4.1114, grad_fn=<MeanBackward0>)
tensor(4.1145, grad_fn=<MeanBackward0>)


In [ ]:
from torchsummary import summary
summary(resnet,(3,160,160))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 79, 79]             864
       BatchNorm2d-2           [-1, 32, 79, 79]              64
              ReLU-3           [-1, 32, 79, 79]               0
       BasicConv2d-4           [-1, 32, 79, 79]               0
            Conv2d-5           [-1, 32, 77, 77]           9,216
       BatchNorm2d-6           [-1, 32, 77, 77]              64
              ReLU-7           [-1, 32, 77, 77]               0
       BasicConv2d-8           [-1, 32, 77, 77]               0
            Conv2d-9           [-1, 64, 77, 77]          18,432
      BatchNorm2d-10           [-1, 64, 77, 77]             128
             ReLU-11           [-1, 64, 77, 77]               0
      BasicConv2d-12           [-1, 64, 77, 77]               0
        MaxPool2d-13           [-1, 64, 38, 38]               0
           Conv2d-14           [-1, 80,

In [24]:
with open('resent.pkl','wb') as f:
  torch.save(resnet, f)